In [ ]:
import seaborn as sns

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Data import

In [ ]:
df = pd.read_csv('df_initial.csv')

C:\Users\norac\AppData\Local\Temp\ipykernel_5832\3474879787.py:1: DtypeWarning: Columns (9,11,12,13,14,15,16,17,18,19,36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('df_initial.csv')


In [ ]:
df.head()

,Unnamed: 0,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,...,A,40.036944,-121.005833,5.0,USFS,CA,63.0,63.0,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,A,38.933056,-120.404444,5.0,USFS,CA,61.0,61.0,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17.0,17.0,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,A,38.559167,-119.913333,5.0,USFS,CA,3.0,3.0,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,A,38.559167,-119.933056,5.0,USFS,CA,3.0,3.0,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


### Grouping by season

We'll create a season function to group lights on and off by season. We'll use the following mapping:

1: Summer: June/July/August
2: Autumn: September/October/November
3: Winter: December/January/February
4: Spring: March/April/May

In [ ]:
def season(x):
    if 5 < x < 9:
        return '1'
    if 8 < x < 12:
        return '2'
    if x in [1,2,12]:
        return '3'
    if x in [3,4,5]:
        return '4'

Before applying the formula, we're going to transform the format of the discovery and extinction dates and extract the month and days of the week. Note that the variable 'CONT_DATE' contains 47% N/A. Next, we'll look at the duration of the fires and see if we can manage these N/As by using the statistics specific to the duration of the fires.

In [ ]:
import datetime as datetime

#transform formats :

df['START_DATE'] = pd.to_datetime(df['DISCOVERY_DATE']- pd.Timestamp(0).to_julian_date(), unit='D')
df['END_DATE'] = pd.to_datetime(df['CONT_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')

In [ ]:
df['START_MONTH'] = pd.DatetimeIndex(df['START_DATE']).month
df['START_DAY_OF_WEEK'] = pd.DatetimeIndex(df['START_DATE']).weekday

In [ ]:
df['SEASON']=df['START_MONTH'].apply(season)

### Grouping by causes

We'll create a Human_Natural function to define whether the cause of the fire was natural, deliberate human or unintentional human. We will create a category to group causes that are not defined (~8% of the data). We will use the following mapping:

1: natural origin
2: voluntary human origin
3: involuntary human origin
0: unknown origin

In [ ]:
df['STAT_CAUSE_DESCR'].value_counts(normalize=True)

Debris Burning       0.228150
Miscellaneous        0.172194
Arson                0.149673
Lightning            0.148085
Missing/Undefined    0.088661
Equipment Use        0.078498
Campfire             0.040489
Children             0.032528
Smoking              0.028115
Railroad             0.017791
Powerline            0.007683
Fireworks            0.006116
Structure            0.002019
Name: STAT_CAUSE_DESCR, dtype: float64

In [ ]:
def Human_Natural(x):
  if x in ['Missing/Undefined']:
    return 0
  if x in ['Lightning']:
    return 1
  if x in ['Arson']:
    return 2
  else:
    return 3

In [ ]:
df['CAUSE_CATEG'] = df['STAT_CAUSE_DESCR'].apply(Human_Natural)

We will classify fires according to their size to define a Mega Fire class for fires that have burnt more than 24,000 acres (around 10,000 hectares). We will use the following mapping:

0: standard fires
1: mega fires

In [ ]:
df['MEGA_FEUX'] = df['FIRE_SIZE'].apply(lambda x: 1 if x > 24710 else 0)

### Regroupement en régions

In [ ]:
regions = []
North_East = ['CT','ME','MA','NH','VT','NJ','NY','RI','PA']
Midwest = ['MI','IL','IN','OH','WI','IA','KS','MN','MO','SD','ND','NE']
South = ['DE','FL','GA','MD','NC','SC','VA','DC','WV','AL','KY','MS','TN','AR','LA','TX','OK']
West = ['AZ','CO','MT','ID','NV','NM','WY','UT','CA','AK','HI','OR','WA']

divisions = []
New_England = ['CT','ME','MA','NH','VT','RI']
Mid_Atlantic = ['NJ','NY','PA']
East_North_Central = ['MI','IL','IN','OH','WI']
West_North_Central = ['IA','KS','MN','MO','SD','ND','NE']
South_Atlantic = ['DE','FL','GA','MD','NC','SC','VA','DC','WV']
East_South_Central = ['AL','KY','MS','TN']
West_South_Central = ['AR','LA','TX','OK']
Mountain = ['AZ','CO','MT','ID','NV','NM','WY','UT']
Pacific = ['CA','AK','HI','OR','WA']


def func_regions(series):
    for row in series:
        if row in North_East:
            regions.append('North_East')
        elif row in Midwest:
            regions.append('Midwest')
        elif row in South:
            regions.append('South')
        elif row in West:
            regions.append('West')
        else:
            regions.append('Other')

func_regions(df['STATE'])

def func_divisions(series):
    for row in series:
        if row in New_England:
            divisions.append('North East - New England')
        elif row in Mid_Atlantic:
            divisions.append('North East - Mid Atlantic')
        elif row in East_North_Central:
            divisions.append('Midwest - East North Central')
        elif row in West_North_Central:
            divisions.append('Midwest - West North Central')
        elif row in South_Atlantic:
            divisions.append('South- South Atlantic')
        elif row in East_South_Central:
            divisions.append('South - East South Central')
        elif row in West_South_Central:
            divisions.append('South - West South Central')
        elif row in Mountain:
            divisions.append('West - Mountain')
        elif row in Pacific:
            divisions.append('West - Pacific')
        else:
            divisions.append('Other')

func_divisions(df['STATE'])

In [ ]:
df['REGIONS'] = regions
df['DIVISIONS'] = divisions

### Cleaned dataframe

In the next step, we're going to delete the columns we won't be using in the analysis.

In [ ]:
to_drop = ['Unnamed: 0','STAT_CAUSE_CODE','STAT_CAUSE_DESCR','FOD_ID','FPA_ID','SOURCE_SYSTEM','SOURCE_SYSTEM_TYPE','NWCG_REPORTING_AGENCY','NWCG_REPORTING_UNIT_ID','NWCG_REPORTING_UNIT_NAME','SOURCE_REPORTING_UNIT','SOURCE_REPORTING_UNIT_NAME',
           'LOCAL_FIRE_REPORT_ID','LOCAL_INCIDENT_ID','FIRE_CODE','FIRE_NAME','ICS_209_INCIDENT_NUMBER','ICS_209_NAME','MTBS_ID','MTBS_FIRE_NAME','COMPLEX_NAME',
           'DISCOVERY_DOY','CONT_DOY','FIPS_CODE','FIPS_NAME','Shape','DISCOVERY_DATE','CONT_DATE','START_DATE','END_DATE','CONT_TIME','OWNER_CODE','OWNER_DESCR']
df_clean = df.drop(columns=to_drop)
df_clean.set_index(df_clean.OBJECTID,inplace=True)
df_clean.drop(columns='OBJECTID',inplace=True)
df_clean.head()

,FIRE_YEAR,DISCOVERY_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,START_MONTH,START_DAY_OF_WEEK,SEASON,CAUSE_CATEG,MEGA_FEUX,REGIONS,DIVISIONS
OBJECTID,,,,,,,,,,,,,,,
1,2005,1300.0,0.10,A,40.036944,-121.005833,CA,63.0,2,2,3,3,0,West,West - Pacific
2,2004,845.0,0.25,A,38.933056,-120.404444,CA,61.0,5,2,4,1,0,West,West - Pacific
3,2004,1921.0,0.10,A,38.984167,-120.735556,CA,17.0,5,0,4,3,0,West,West - Pacific
4,2004,1600.0,0.10,A,38.559167,-119.913333,CA,3.0,6,0,1,1,0,West,West - Pacific
5,2004,1600.0,0.10,A,38.559167,-119.933056,CA,3.0,6,0,1,1,0,West,West - Pacific


In [ ]:
df_clean[df_clean['STATE']=='NY'].head()

,FIRE_YEAR,DISCOVERY_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,START_MONTH,START_DAY_OF_WEEK,SEASON,CAUSE_CATEG,MEGA_FEUX,REGIONS,DIVISIONS
OBJECTID,,,,,,,,,,,,,,,
84589,1995,1430.0,1.0,B,42.533333,-76.733333,NY,NaN,8,6,1,3,0,North_East,North East - Mid Atlantic
84590,1995,1430.0,0.2,A,42.533333,-76.806944,NY,NaN,8,6,1,3,0,North_East,North East - Mid Atlantic
84591,1995,1700.0,0.2,A,42.441111,-76.811389,NY,NaN,9,1,2,2,0,North_East,North East - Mid Atlantic
109803,1998,2101.0,5.0,B,42.588333,-76.821667,NY,NaN,4,0,4,3,0,North_East,North East - Mid Atlantic
109804,1998,1631.0,1.0,B,42.561111,-76.782500,NY,NaN,5,1,4,1,0,North_East,North East - Mid Atlantic


In [ ]:
df_clean['STATE'].value_counts()

CA    189550
GA    168867
TX    142021
NC    111277
FL     90261
SC     81315
NY     80870
MS     79230
AZ     71586
AL     66570
OR     61088
MN     44769
OK     43239
MT     40767
NM     37478
ID     36698
CO     34157
WA     33513
WI     31861
AR     31663
TN     31154
SD     30963
UT     30725
LA     30013
KY     27089
NJ     25949
PR     22081
WV     21967
VA     21833
MO     17953
NV     16956
ND     15201
WY     14166
ME     13150
AK     12843
MI     10502
HI      9895
PA      8712
NE      7973
KS      7673
CT      4976
IA      4134
MD      3622
OH      3479
MA      2626
NH      2452
IL      2327
IN      2098
RI       480
VT       456
DE       171
DC        66
Name: STATE, dtype: int64

In [ ]:
df_clean.to_csv(r'C:\Users\norac\OneDrive\Documents\Reconversion\DataScientest\Projet Feux de Forêt\df_clean.csv',index=False)